<center>
<h4>CDS 110, Lecture 6b</h4>
<font color=blue><h1>Trajectory Tracking for a Kinematic Car</h1></font>
<h3>Richard M. Murray, Winter 2024</h3>
</center>
<br>

This notebook contains an example of using trajectory tracking for a (nonlinear) state space system.  The controller is of the form

$$
  u = u_\text{d} − K (x − x_\text{d}),
$$

where $x_\text{d}, u_\text{d}$ is a feasible trajectory, and $K$ is a feedback gain first computed around a nominal condition and then computed using gain scheduling.

In [ ]:
# Import the packages needed for the examples included in this notebook
import numpy as np
import matplotlib.pyplot as plt
import itertools
from cmath import sqrt
from math import pi

try:
  import control as ct
  print("python-control version:", ct.__version__)
except ImportError:
  # Version 0.10.0 is enough for this notebook
  !pip install control
  import control as ct

# Import the python optimal-control module
import control.optimal as opt
import control.flatsys as fs

## Vehicle Steering Dynamics

The vehicle dynamics are given by a simple bicycle model:

<table>
<tr>
    <td width="50%"><img src="https://fbswiki.org/wiki/images/5/52/Kincar.png" width=480></td>
    <td width="50%">
$$\large
\begin{aligned}
  \dot x &= \cos\theta\, v \\
  \dot y &= \sin\theta\, v \\
  \dot\theta &= \frac{v}{l} \tan \delta
\end{aligned}
$$
    </td>
</tr>
</table>

We take the state of the system as $(x, y, \theta)$ where $(x, y)$ is the position of the vehicle in the plane and $\theta$ is the angle of the vehicle with respect to horizontal.  The vehicle input is given by $(v, \delta)$ where $v$ is the forward velocity of the vehicle and $\delta$ is the angle of the steering wheel.  The model includes saturation of the vehicle steering angle.

In [ ]:
# Vehicle steering dynamics
#
# System state: x, y, theta
# System input: v, delta
# System output: x, y
# System parameters: wheelbase, maxsteer
#
try:
  from kincar import kincar, plot_lanechange
except ImportError:
  !wget --no-check-certificate https://www.cds.caltech.edu/~murray/courses/cds110/sp2024/kincar.py
  from kincar import kincar, plot_lanechange

print(kincar)
print(kincar.params)

The following code is contained in the model:
```
def _kincar_update(t, x, u, params):
    # Get the parameters for the model
    l = params['wheelbase']             # vehicle wheelbase
    deltamax = params['maxsteer']         # max steering angle (rad)

    # Saturate the steering input
    delta = np.clip(u[1], -deltamax, deltamax)

    # Return the derivative of the state
    return np.array([
        np.cos(x[2]) * u[0],            # xdot = cos(theta) v
        np.sin(x[2]) * u[0],            # ydot = sin(theta) v
        (u[0] / l) * np.tan(delta)      # thdot = v/l tan(delta)
    ])
```

## State feedback controller

We start by designing a state feedback controller that can be used to stabilize the system.  We design the controller around a nominal forward speed of 10 m/s and then apply this to the vehicle at different speeds.

In [ ]:
# Compute the linearization of the dynamics at a nominal point
x_nom = np.array([0, 0, 0])
u_nom = np.array([5, 0])
P = ct.linearize(kincar, x_nom, u_nom)    # Linearized systems
print(P)

Qx = np.diag([1, 10, 0.1])
Qu = np.diag([1, 1])
K, _, _ = ct.lqr(P.A, P.B, Qx, Qu)
print(K)

In [ ]:
# Create the closed loop system using create_statefbk_iosystem
?ct.create_statefbk_iosystem
ctrl, clsys = ct.create_statefbk_iosystem(
    kincar, K, xd_labels=['xd', 'yd', 'thetad'], ud_labels=['vd', 'deltad'])
print(clsys)

In [ ]:
# Create a trajectory corresponding to a slow lane change
x0 = np.array([0, -2, 0]); u0 = [10, 0]
xf = np.array([100, 2, 0])
Tf = 10
timepts = np.linspace(0, Tf, 20)

straight_line = (               # straight line from start to end with nominal input
    np.array([x0 + (xf - x0) * t/Tf for t in timepts]).transpose(),
    u0
)

desired = opt.solve_ocp(
    kincar, timepts, x0,
    cost=opt.quadratic_cost(kincar, None, Qu, u0=u0),
    terminal_constraints=opt.state_range_constraint(kincar, xf, xf),
    initial_guess=straight_line)

plot_lanechange(desired.time, desired.states, desired.inputs, yf=xf)

In [ ]:
# Simulate the system with an initial condition error
# Use t_eval to evaluate at points between inputs
actual = ct.input_output_response(
    clsys, timepts, [desired.states, desired.inputs],
    X0=[-3, -5, 0], t_eval=np.linspace(0, Tf, 500))

plot_lanechange(actual.time, actual.states, actual.outputs[3:])
plot_lanechange(desired.time, desired.states, desired.inputs, yf=xf)

Note that the value of $\delta$ is very large at the start.  This is truncated in the model so that it does not exceed $\pm 0.5$ rad.

## Reference trajectory subsystem

In addition to generating a trajectory for the system, we can also create $x_\text{d}$ and $u_\text{d}$ corresponding to reference inputs $r_y$ and $r_v$.

The reference trajectory block below generates a simple trajectory for the system given the desired speed (vref) and lateral position (yref).  The trajectory consists of a straight line of the form (vref * t, yref, 0) with nominal
input (vref, 0).

In [ ]:
# System state: none
# System input: vref, yref
# System output: xd, yd, thetad, vd, deltad
# System parameters: none
#
def trajgen_output(t, x, u, params):
    vref, yref = u
    return np.array([vref * t, yref, 0, vref, 0])

# Define the trajectory generator as an input/output system
trajgen = ct.nlsys(
    None, trajgen_output, name='trajgen',
    inputs=('vref', 'yref'),
    outputs=('xd', 'yd', 'thetad', 'vd', 'deltad'))

print(trajgen)

## Step responses

To explore the dynamics of the system, we create a set of lane changes at different forward speeds.  Since the linearization depends on the speed, this means that the closed loop performance of the system will vary.

In [ ]:
steering_fixed = ct.interconnect(
    [kincar, ctrl, trajgen],
    inputs=['vref', 'yref'],
    outputs=kincar.output_labels + kincar.input_labels
)
print(steering_fixed)

In [ ]:
# Set up the simulation conditions
yref = 1
T = np.linspace(0, 5, 100)

# Do an iteration through different speeds
for vref in [2, 5, 20]:
    # Simulate the closed loop controller response
    tout, yout = ct.input_output_response(
        steering_fixed, T, [vref * np.ones(len(T)), yref * np.ones(len(T))],
        params={'maxsteer': 1})

    # Plot the results
    plot_lanechange(tout, yout, yout[3:])

# Label the different curves
plt.subplot(3, 1, 1)
plt.legend(["$v_d$ = " + f"{vref}" for vref in [2, 10, 20]])
plt.tight_layout()

## Gain scheduled controller

For this system we use a simple schedule on the forward vehicle velocity and
place the poles of the system at fixed values.  The controller takes the
current and desired vehicle position and orientation plus the velocity
velocity as inputs, and returns the velocity and steering commands.

Linearizing the system about the desired trajectory, we obtain

$$
  \begin{aligned}
    A(x_\text{d}) &= \left. \frac{\partial f}{\partial x} \right|_{(x_\text{d}, u_\text{d})}
      = \left.
        \begin{bmatrix}
          0 & 0 & -\sin\theta_\text{d}\, v_\text{d} \\ 0 & 0 & \cos\theta_\text{d}\, v_\text{d} \\ 0 & 0 & 0
        \end{bmatrix}
        \right|_{(x_\text{d}, u_\text{d})}
      = \begin{bmatrix}
          0 & 0 & 0 \\ 0 & 0 & v_\text{d} \\ 0 & 0 & 0
         \end{bmatrix}, \\
    B(x_\text{d}) &= \left. \frac{\partial f}{\partial u} \right|_{(x_\text{d}, u_\text{d})}
     = \begin{bmatrix}
       1 & 0 \\ 0 & 0 \\ 0 & v_\text{d}/l
       \end{bmatrix}.
  \end{aligned}
$$

We see that these matrices depend only on $\theta_\text{d}$ and $v_\text{d}$, so we choose these as the scheduling variables and design a controller of the form

$$
u = u_\text{d} - K(\mu) (x - x_\text{d})
$$

where $\mu = (\theta_\text{d}, v_\text{d})$ and we interpolate the gains based on LQR controllers computed at a fixed set of points $\mu_i$.

In [ ]:
# Define the points for the scheduling variables
gs_speeds = [2, 10, 20]
gs_angles = np.linspace(-pi, pi, 4)

# Create controllers at each scheduling point (
points = [np.array([speed, angle])
          for speed in gs_speeds for angle in gs_angles]
gains = [np.array(ct.lqr(kincar.linearize(
    [0, 0, angle], [speed, 0]), Qx, Qu)[0])
    for speed in gs_speeds for angle in gs_angles]
print(f"{points=}")
print(f"{gains=}")

# Create the gain scheduled system
ctrl_gs, _ = ct.create_statefbk_iosystem(
    kincar, (gains, points), name='controller',
    xd_labels=['xd', 'yd', 'thetad'], ud_labels=['vd', 'deltad'],
    gainsched_indices=['vd', 'theta'], gainsched_method='linear')
print(ctrl_gs)

## System construction

The input to the full closed loop system is the desired lateral position and the desired forward velocity.  The output for the system is taken as the full vehicle state plus the velocity of the vehicle.

We construct the system using the `ct.interconnect` function and use signal labels to keep track of everything.  

In [ ]:
steering_gainsched = ct.interconnect(
    [trajgen, ctrl_gs, kincar], name='steering',
    inputs=['vref', 'yref'],
    outputs=kincar.output_labels + kincar.input_labels
)
print(steering_gainsched)

## System simulation

We now simulate the gain scheduled controller for a step input in the $y$ position, using a range of vehicle speeds $v_\text{d}$:

In [ ]:
# Plot the reference trajectory for the y position
# plt.plot([0, 5], [yref, yref], 'k-', linewidth=0.6)

# Find the signals we want to plot
y_index = steering_gainsched.find_output('y')
v_index = steering_gainsched.find_output('v')

# Do an iteration through different speeds
for vref in [2, 5, 20]:
    # Simulate the closed loop controller response
    tout, yout = ct.input_output_response(
        steering_gainsched, T, [vref * np.ones(len(T)), yref * np.ones(len(T))],
        X0=[0, 0, 0], params={'maxsteer': 0.5}
    )

    # Plot the results
    plot_lanechange(tout, yout, yout[3:])

# Label the different curves
plt.subplot(3, 1, 1)
plt.legend(["$v_d$ = " + f"{vref}" for vref in [2, 10, 20]])
plt.tight_layout()